In [Part 1](https://llefebure.github.io/articles/2019-06/prescribing-behavior-part1), we did some exploratory analysis on the Roam Analytics Medicare Part D data with an eye towards predicting a provider's specialty from their prescribing behavior. To summarize, the key observations for the classification problem were as follows:

* Highly imbalanced classes
* Widely varying levels of expected separation between classes
* Drug counts follow a power law distribution, so a TFIDF transformation could be useful
* Some features are very sparse
* Strong collinearity in the features

This post (Part 2) focuses on exploring a few models for the multiclass classification task itself.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.metrics.base
import warnings
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import (
    StratifiedKFold, GridSearchCV, train_test_split, cross_val_predict)
pd.options.display.max_columns = None
%matplotlib inline

In [3]:
provider_variables = pd.read_csv('data/provider_variables.csv')
vectorizer = np.load('data/vectorizer.npy')[()]
X = np.load('data/X.npy')[()]

## Evaluation Strategy

To compare performance across models, we split the data into separate training and test sets and stratify by specialty to ensure that both partitions have a representative class distribution

We will look at the F1 metric to gauge performance. In the multiclass classifcation case, this metric can take different forms. For example, we can compute the F1 score for each class separately and average the resulting values, or we can sum up the true positive, false positive, etc. counts across all classes and compute a global F1 score. The former implicitly weights each class equally despite their unequal sizes, while the latter favors good performance on the larger classes.

Finally, the F1 metric can be undefined for some classes in the case where there are no predicted values for a class, so we ignore the resulting error.

In [4]:
warnings.filterwarnings(
    'ignore', category=sklearn.exceptions.UndefinedMetricWarning)

In [5]:
provider_variables_train, provider_variables_test, X_train, X_test = \
    train_test_split(
        provider_variables, X, test_size=.2, stratify=provider_variables.specialty)

## Model Building

### Features

First, we transform the features with TFIDF to upweight the rarer and potentially more distinguishing features while downweighting the most popular features.

In [41]:
tfidf_transformer = TfidfTransformer()
tfidf_transformer.fit(X_train)
X_train_tfidf = tfidf_transformer.transform(X_train)
X_test_tfidf = tfidf_transformer.transform(X_test)

### Logistic Regression

Now that we have features, we build a multinomial logistic regression model. This is a simple model that is easy to train, so it will give us a good baseline level of understanding on the nuances of this problem. The collinearity in the features is problematic if you want to interpret the model coefficients, but it shouldn't affect performance.

In [42]:
lr_model = GridSearchCV(
    LogisticRegression(
        multi_class='multinomial',
        solver='lbfgs'
    ),
    scoring=['f1_macro', 'f1_micro'],
    param_grid={'C': np.logspace(0, 10, 10)},
    return_train_score=True,
    refit='f1_macro'
)
lr_model.fit(X_train_tfidf, provider_variables_train.specialty)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': array([1.00000e+00, 1.29155e+01, 1.66810e+02, 2.15443e+03, 2.78256e+04,
       3.59381e+05, 4.64159e+06, 5.99484e+07, 7.74264e+08, 1.00000e+10])},
       pre_dispatch='2*n_jobs', refit='f1_macro', return_train_score=True,
       scoring=['f1_macro', 'f1_micro'], verbose=0)

In [43]:
cv_preds = cross_val_predict(
    lr_model.best_estimator_, X_train_tfidf, provider_variables_train.specialty)

Looking at the individual per class F1 scores, we see a pretty wide range of values. While the larger classes tend to have higher scores, the relationship is not perfect. We also see that the observations from Part 1 that some of the well separated classes such as Neurology and Rheumatology should be fairly easy to predict are indeed true as these classes have an F1 of over .95.

In looking at the confusion matrix of the cross validated predictions, we can definitely see patterns in the errors for the low performing classes. The biggest issue appears to be distinguishing between very similar classes rather than failing completely on strictly small sample classes. For example:

* Child & Adolescent Psychiatry getting mistaken for Psychiatry.
* Acute Care getting mistaken for Family.
* Primary Care getting mistaken for Family.
* Interventional Cardiology getting mistaken for Cardiovascular Disease.

In [44]:
class_labels = provider_variables_train.specialty.value_counts()
f1_scores = f1_score(
    provider_variables_train.specialty, cv_preds, labels=class_labels.index,
    average=None)
class_summary = pd.DataFrame(
    list(zip(class_labels.index, f1_scores, class_labels.values,
             np.argsort(-f1_scores).argsort() + 1)),
    columns=['Specialty', 'F1', 'True Count', 'F1 Rank']
)
class_summary

,Specialty,F1,True Count,F1 Rank
0,Cardiovascular Disease,0.898652,4249,6
1,Family,0.643547,3999,12
2,Psychiatry,0.914622,2433,5
3,Medical,0.240791,1650,17
4,Geriatric Medicine,0.654655,1544,11
5,Nephrology,0.931343,1516,4
6,Neurology,0.951421,1393,2
7,"Endocrinology, Diabetes & Metabolism",0.934723,1116,3
8,Adult Health,0.137077,756,20
9,Rheumatology,0.953229,683,1


In [55]:
confusion_matrix_cv_preds = pd.DataFrame(
    confusion_matrix(provider_variables_train.specialty, cv_preds,
                     labels=class_labels.index))
confusion_matrix_cv_preds.index = class_labels.index
confusion_matrix_cv_preds.columns = class_labels.index
confusion_matrix_cv_preds # element (i,j) refers to number in class i predicted to be in class j

,Cardiovascular Disease,Family,Psychiatry,Medical,Geriatric Medicine,Nephrology,Neurology,"Endocrinology, Diabetes & Metabolism",Adult Health,Rheumatology,Pulmonary Disease,Infectious Disease,Adult Medicine,Hematology & Oncology,Gastroenterology,Interventional Cardiology,Psych/Mental Health,Gerontology,Primary Care,Pain Medicine,Adolescent Medicine,Acute Care,Sports Medicine,Interventional Pain Medicine,Child & Adolescent Psychiatry,Addiction Medicine,Critical Care Medicine,Medical Oncology,Clinical Cardiac Electrophysiology
Cardiovascular Disease,3999,40,1,26,50,9,1,2,14,1,7,3,20,3,4,52,0,1,0,0,3,3,0,0,0,0,0,0,10
Family,59,2867,26,531,123,27,12,19,139,6,18,38,47,2,5,2,8,23,10,7,8,7,9,4,0,1,0,0,1
Psychiatry,0,11,2330,7,2,1,7,0,3,0,0,0,0,0,0,0,68,0,0,3,0,0,0,0,1,0,0,0,0
Medical,38,942,11,353,65,9,23,15,62,8,10,30,35,0,4,1,0,7,8,10,8,3,6,2,0,0,0,0,0
Geriatric Medicine,45,144,2,47,1090,13,1,3,43,0,25,13,55,5,6,1,0,33,1,0,11,0,3,0,0,2,1,0,0
Nephrology,18,22,0,11,25,1404,0,2,8,1,9,4,5,3,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0
Neurology,1,14,22,17,3,0,1322,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,1,3,0,2,0,0,0
"Endocrinology, Diabetes & Metabolism",6,22,1,15,23,2,0,1031,6,0,0,0,6,0,0,0,0,0,0,0,2,0,2,0,0,0,0,0,0
Adult Health,46,364,8,85,60,14,4,13,83,1,7,22,12,1,3,0,0,20,4,2,1,1,4,1,0,0,0,0,0
Rheumatology,2,7,0,12,12,0,0,0,3,642,2,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


Finally, we will obtain the F1 scores on the test data.

In [60]:
print('Macro F1: ', f1_score(
    provider_variables_test.specialty, lr_model.predict(X_test_tfidf),
    average='macro'))
print('Micro F1: ', f1_score(
    provider_variables_test.specialty, lr_model.predict(X_test_tfidf),
    average='micro'))

Macro F1:  0.4571918561271133
Micro F1:  0.7372719374456995


### XGBoost

For the sake of having something to compare the logistic regression model against, we train a vanilla XGBoost model with the default parameters. Performance is pretty comparable to that of the Logistic Regression. It underperforms on the Macro F1 and outperforms on the Micro F1, showing that the model is biased towards better performance on the larger classes while ignoring smaller ones.

In [64]:
xgb_model = XGBClassifier()
xgb_model.fit(X_train_tfidf, provider_variables_train.specialty)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [68]:
preds = model.predict(X_test_tfidf)

/Users/lukelefebure/envs/python_3.6.3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [67]:
print('Macro F1: ', f1_score(
    provider_variables_test.specialty, xgb_model.predict(X_test_tfidf),
    average='macro'))
print('Micro F1: ', f1_score(
    provider_variables_test.specialty, xgb_model.predict(X_test_tfidf),
    average='micro'))

/Users/lukelefebure/envs/python_3.6.3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Macro F1:  0.4193069563936367
Micro F1:  0.7449174630755866


/Users/lukelefebure/envs/python_3.6.3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


## Conclusion

The most interesting finding here for me was that the class imbalance really didn't seem to be the biggest problem, rather the similarity between certain classes was. The model had very good accuracy on small sample specialized domains such as Hematology & Oncology but struggled with the ones that had highly related classes such as Child & Adolescent Psychiatry.

I didn't really do any model tuning here, though if this were a problem worth pursuing further, I think my next steps would be to look more closely at the lower performing classes. For example, if we tried to build a binary classifier for Child & Adolescent Psychiatry vs. Pyschiatry, would there be any distinguishing features between the two?